In [1]:
# from IPython.display import clear_output
# import openml # openML
import pandas as pd
import numpy as np
from os import listdir # get dir list
from os.path import isfile, join # get dir list
import random
# from pymfe.mfe import MFE

In [2]:
def getFolderFilesList(path):
    return [filename for filename in listdir(path) if isfile(join(path, filename))]

In [3]:
dictOfFIles = {'DT': [], 'RF': [], 'KNN': [] }
filtersList = getFolderFilesList('Results')
for i in filtersList:
    if (i.split('_')[-1] == 'resultsDT.csv.zip'):
        dictOfFIles['DT'].append(i)
    elif (i.split('_')[-1] == 'resultsRF.csv.zip'):
        dictOfFIles['RF'].append(i)
    else:
        dictOfFIles['KNN'].append(i)

metaFeatures = pd.read_csv('meta_features_dropna.csv')
metaFeatures = metaFeatures.set_index('Dataset').sort_index()
# metaFeatures = metaFeatures.drop(['lh_trace', 'roy_root'], axis = 1)
metabase = {'DT': metaFeatures.copy(), 'RF': metaFeatures.copy(), 'KNN': metaFeatures.copy() }
for value in dictOfFIles:
    for file in dictOfFIles[value]:
        df = pd.read_csv(f'Results/{file}')
        filterName = file.split('_')[1]
        df = df.rename({'Unnamed: 0': 'Dataset', 'test_f1_macro_mean': f'test_f1_macro_mean_{filterName}'},  axis=1)
        df = df.set_index('Dataset')
        df = df.sort_index()
        metabase[value] = metabase[value].join(df[f'test_f1_macro_mean_{filterName}'])

In [4]:
metabase['DT']

,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,best_node.mean,best_node.sd,can_cor.mean,cat_to_num,class_conc.mean,...,test_f1_macro_mean_dynamicCF,test_f1_macro_mean_edgeBoostFilter-10,test_f1_macro_mean_HARF-80,test_f1_macro_mean_AENN-3,test_f1_macro_mean_HARF-90,test_f1_macro_mean_ORBoost-7,test_f1_macro_mean_HARF-70,test_f1_macro_mean_GE-5,test_f1_macro_mean_AENN-5,test_f1_macro_mean_GE-11
Dataset,,,,,,,,,,,,,,,,,,,,,
1004_synthetic_control_10_0.csv.gz,8.331300e-02,0.062438,2.999999,0.000006,0.1000,0.878333,0.076965,0.688118,0.0,0.012346,...,0.964651,0.785559,0.788595,0.886801,0.511807,0.942136,0.940946,0.811558,0.870598,0.879085
1004_synthetic_control_10_1.csv.gz,1.005420e-01,0.057744,2.999999,0.000006,0.1000,0.885000,0.084451,0.707928,0.0,0.012193,...,0.964829,0.559755,0.903792,0.792726,0.724532,0.869834,0.925005,0.899262,0.848646,0.913513
1004_synthetic_control_10_2.csv.gz,8.331762e-02,0.067932,2.999999,0.000006,0.1000,0.875000,0.087995,0.729191,0.0,0.012570,...,0.970423,0.542222,0.790358,0.887069,0.650441,0.962819,0.964298,0.843487,0.900485,0.903766
1004_synthetic_control_10_3.csv.gz,6.843893e-02,0.060531,2.999999,0.000006,0.1000,0.861667,0.078969,0.718893,0.0,0.012131,...,0.911309,0.698734,0.878177,0.870206,0.659445,0.951542,0.938964,0.890814,0.861553,0.912285
1004_synthetic_control_10_4.csv.gz,6.317691e-02,0.059250,2.999999,0.000006,0.1000,0.870000,0.083458,0.718633,0.0,0.012191,...,0.927307,0.675416,0.926960,0.862662,0.578659,0.929083,0.948295,0.879535,0.906637,0.875103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997_balance-scale_5_5.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.512000,0.043451,0.709977,0.0,0.032749,...,0.651579,0.487275,0.578027,0.722445,0.588258,0.630931,0.606106,0.596068,0.735181,0.619538
997_balance-scale_5_6.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.507200,0.040635,0.711547,0.0,0.032597,...,0.653752,0.498621,0.595954,0.725088,0.600003,0.640562,0.611638,0.610145,0.701347,0.624082
997_balance-scale_5_7.csv.gz,-6.250002e-08,0.000000,2.321928,0.000000,0.0064,0.504000,0.035327,0.729005,0.0,0.034303,...,0.709290,0.549880,0.571862,0.733404,0.583277,0.626800,0.663737,0.584212,0.736138,0.625725


In [5]:
for value in metabase:
    for i in metabase[value].columns:
        if (metabase[value][i].isin([np.inf, -np.inf]).sum() > 0):
            metabase[value].drop(i, axis=1, inplace=True)
    metabase[value].fillna(0, inplace=True)
    metabase[value].to_csv(f'Metabases/Metabase_{value}.csv.zip')